In [10]:
import os
from pydantic import Extra
import requests
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from langchain.prompts import PromptTemplate
# Run chain
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_ollama import OllamaEmbeddings # type: ignore
import warnings
warnings.filterwarnings('ignore')

In [11]:
os.environ["token"] = "Bearer token|c8ae2a1e-25f9-4bb6-9d1d-35bf46751c0b|d6c5c351530b57d8f3e782c98a47c3ba3e8e6314352ea89b0f028ef2339fef52"
token = os.environ["token"]
Path = r"D:\OneDrive - Wipro\Desktop\Static Dumps\Role-Skill-Cluster Master Report.csv"

In [12]:
from typing import ClassVar

parser = StrOutputParser()

class LlamaLLM(LLM):
    llm_url: ClassVar[str] = 'https://api.waip.wiprocms.com/v1.1/skills/completion/query'
    
    backend:        Optional[str]   = 'gpt-35-turbo-16k'
    temp:           Optional[float] = 0.7
    top_p:          Optional[float] = 0.1
    top_k:          Optional[int]   = 40
    n_batch:        Optional[int]   = 8
    n_threads:      Optional[int]   = 4
    n_predict:      Optional[int]   = 256
    max_tokens:     Optional[int]   = 256
    repeat_last_n:  Optional[int]   = 64
    repeat_penalty: Optional[float] = 1.18

    class Config:
        extra = Extra.forbid

    @property
    def _llm_type(self) -> str:
        return "gpt-35-turbo-16k"
    
    @property
    def _get_model_default_parameters(self):
        return {
            "max_tokens": self.max_tokens,
            #"n_predict": self.n_predict,
            "top_k": self.top_k,
            "top_p": self.top_p,
            "temperature": self.temp,
            #"n_batch": self.n_batch,
            #"repeat_penalty": self.repeat_penalty,
            #"repeat_last_n": self.repeat_last_n,
        }

    def _call(
        self,
        prompt: str,
        user: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        payload = {
        "messages": [
            {
            "content": prompt,
            "role": user
            }
        ],
        "skill_parameters": {
            "model_name": "gpt-35-turbo-16k",
            "max_output_tokens": 4096,
            "temperature": 0,
            "top_k": 5
        },
        "stream_response": False
        }

        headers = {"Content-Type": "application/json","Authorization": token}

        response = requests.post(self.llm_url, json=payload, headers=headers, verify=False)

       # print("API Response:", response.json())
        response.raise_for_status()

        return response.json()  # get the response from the API

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "llmUrl": self.llm_url,
            'model_parameters': self._get_model_default_parameters
            }

In [13]:
llm = LlamaLLM()

In [29]:
ROLE_CLUSTER_MASTER = pd.read_csv(Path)
ROLE_CLUSTER_MASTER_REQD=ROLE_CLUSTER_MASTER[['CLUSTER_NAME','SKILL_NAME']].head(50)

In [ ]:
ROLE_CLUSTER_MASTER_REQD.head(50)

In [34]:
JD = []
for row in ROLE_CLUSTER_MASTER_REQD.head(50).itertuples():
 #print(row)
 try:
    skills=row.SKILL_NAME
    title=row.CLUSTER_NAME
    company="Wipro"
    user = "system"
    prompt = """ As a skilled Technical Interview Panel with expertise in technology and content writing , your role is to meticulously evaluate skills and title provided and give detailed job description having below points outlined.
                Use the mentioned skills along with proficiency  & title to generate the job description, where L1 means Beginner, L2 means Intermediate, L3 means Advanced, and L4 means Expert,
                Job Role: 
                Give skill specified by user
                Clearly state the role/title of the position and give brief summary of the role/title in 3-4 lines
                
                Key Responsibilities:
                Title/Role will have overall impact on responsibilities
                Utilize expertise in [Key Skill 1] and [title/Role] to [Responsibility 1] .
                Apply advanced knowledge of [Key Skill 2]  to [Responsibility 2].
                Demonstrate proficiency in [Key Skill 3] to [Responsibility 3].
                Utilize [Key Skill 4]  to [Responsibility 4].
                Apply [Key Skill 5] to [Responsibility 5].

                Qualifications and Skills:

                Proficiency in [Key Skill 1] is essential for this role, with a strong focus on [Specific Requirement].
                Advanced knowledge of [Key Skill 2] is highly desirable, with the ability to [Specific Requirement].
                Familiarity with [Key Skill 3] is beneficial, with the capacity to [Specific Requirement].
                Experience in [Key Skill 4] is advantageous, with the ability to [Specific Requirement].
                Proficiency in [Key Skill 5] is preferred, with the capability to [Specific Requirement].

                Experience Requirements:

                A minimum of [Number of Years] years of relevant experience in [Industry/Field].
                Previous experience in [Specific Type of Experience] is beneficial, with a focus on [Relevant Proficiency].
                Demonstrated proficiency in [Key Skill 1] and [Key Skill 2] through [Type of Experience].
                Proven track record of applying [Key Skill 3] and [Key Skill 4] in [Type of Experience].

                """ + skills + title + company # type: ignore

    result = llm._call(prompt,user)
 except Exception as e:
    print("Error:",e)
    continue
 parsed_result = result['data']['content'] # type: ignore

 prompt2 = f"""Step 1: Analyze Current JD: Current Job Description ({parsed_result}) here.
Core Responsibilities: 
[List responsibilities from JD] 
Required Skills:
[List skills from JD] 
Role Expectations: 
[Summarize expectations]   
Step 2: AI-Enabled JD (Rewrite with AI Integration)
For each responsibility, show how AI can enhance it: 
Responsibility: [Original responsibility]
AI Integration: [Explain how AI tools, automation, or data-driven decision-making can improve this task] 
Add AI-related competencies: 
[AI Related competencies based on Original responsibility] 
Step 3: Career Transition Plan
(Assume the role has minimal or no prior AI exposure)
A. Skills to Learn 
Technical: 
[List of Technical skill infused with AI based Original responsibility] 
Soft Skills: 
[List down Soft skill infused with IA based Original responsibility]
B. Recommended Certifications & Tools
Certifications:
[List down recommended certifications infused with AI based Original responsibility]
Tools:
[List down recommended tools infused with AI based Original responsibility]
C. Role Evolution Path
Short-Term (0–6 months):
Learn AI basics and simple automation tools
Apply AI for repetitive tasks
[ Define a short-term goal for the user based on the tools and certifications mentioned earlier. Assume the user is starting from scratch with no prior knowledge of AI. Begin with foundational concepts and align the goal with the user’s original job responsibilities as the base.]
Mid-Term (6–18 months):
[Define a mid-term goal for the user that builds on the foundational AI knowledge acquired in the short-term phase. Incorporate intermediate concepts, practical application of tools, and relevant certifications mentioned earlier. Ensure the goal aligns with the user’s original job responsibilities while gradually expanding their role to include AI-driven decision-making and process optimisation.]
Long-Term (18+ months):
[Define a long-term goal for the user that demonstrates advanced proficiency in AI concepts and tools, building on the foundational and intermediate stages. Focus on strategic integration of AI into business processes, leadership in AI-driven initiatives, and leveraging certifications to position the user as an organisational thought leader. Ensure the goal aligns with the user’s original responsibilities while expanding their influence to drive innovation and transformation across the enterprise.]"""
 summarized_result = llm._call(prompt2,user)
 summarized_result2 = summarized_result['data']['content'] # type: ignore
 JD.append("CLUSTER NAME :"+title +'\n'+"CLUSTER SKILLS : "+skills+'\n'+summarized_result2+'\n'+'\n'+'\n'+'\n') # type: ignore

Error: can only concatenate str (not "float") to str
Error: can only concatenate str (not "float") to str


In [35]:
for item in JD:
    print(item)

CLUSTER NAME :MIS EXECUTIVE L1
CLUSTER SKILLS : Excel (L1), Project Delivery (L1), Reporting (L1), MIS (L1)
Step 1: Analyze Current JD

Core Responsibilities:
- Collect, organize, and maintain accurate data records using Excel and MIS tools.
- Assist in tracking project progress and deliverables with basic project delivery knowledge.
- Generate standard and ad-hoc reports for various stakeholders.
- Monitor key performance indicators (KPIs) to support management decision-making.
- Automate routine tasks and improve data processing efficiency using Excel.

Required Skills:
- Proficiency in Excel (data entry, formulas, basic data analysis).
- Basic knowledge of Project Delivery (understanding timelines and deliverables).
- Familiarity with Reporting (creating clear and concise reports).
- Experience in MIS (data system management and data accuracy).
- Proficiency in general office software and communication tools.

Role Expectations:
- Ensure data integrity and accuracy in MIS.
- Provide

In [36]:
with open("NewJD-19Nov25", "w") as outfile:
    outfile.write("\n".join(JD))